In [ ]:
import boto3 
import json
import os
import copy
import matplotlib.pyplot as plt
import pickle
import cv2
import numpy as np
from io import BufferedReader


recog_result_root_dir = './recognition_results/'
if not os.path.exists(recog_result_root_dir):
    os.makedirs(recog_result_root_dir)

client=boto3.client('rekognition')


def same_face(face_1_bytes, face_2_bytes):
    try:
        response=client.compare_faces(
            SimilarityThreshold=85,
            SourceImage={'Bytes': face_1_bytes},
            TargetImage={'Bytes': face_2_bytes})

        matched = len(response['FaceMatches']) > 0
        return matched
    except Exception as e:
        return False


# rec =  [left, right, bottom, top]
def calculateIOU(rec1, rec2):
    intersect_l = max(rec1[0],rec2[0])
    intersect_r = min(rec1[1],rec2[1])
    intersect_b = max(rec1[2],rec2[2])
    intersect_t = min(rec1[3],rec2[3])
    if intersect_l >= intersect_r or intersect_b >= intersect_t:
        return 0.0
    else:
        S_rect1 = (rec1[1]-rec1[0])*(rec1[3]-rec1[2])
        S_rect2 = (rec2[1]-rec2[0])*(rec2[3]-rec2[2])
        intersect = (intersect_r - intersect_l)*(intersect_t - intersect_b)
        iou = float(intersect) / (S_rect1 + S_rect2 - intersect)
        return iou
    

def identify_already_appear_before(faces_collector, 
                                   faces_in_last_frame, 
                                   compare_face_bytes, 
                                   current_face, 
                                   image_height,
                                   image_width,
                                   attributes):
    
    # 输入人脸的位置
    base_height = int(attributes['BoundingBox']['Height'] * image_height)
    base_left = int(attributes['BoundingBox']['Left'] * image_width)
    base_top = int(attributes['BoundingBox']['Top'] * image_height)
    base_width = int(attributes['BoundingBox']['Width'] * image_width)
    
    # 先基于面积来评估追踪
    max_iou = -1.0
    matched_face_id = -1
    for idx, face_obj in enumerate(faces_in_last_frame):
        height = int(face_obj['attributes']['BoundingBox']['Height'] * image_height)
        left = int(face_obj['attributes']['BoundingBox']['Left'] * image_width)
        top = int(face_obj['attributes']['BoundingBox']['Top'] * image_height)
        width = int(face_obj['attributes']['BoundingBox']['Width'] * image_width)
        
        # 计算over_lap
        iou = calculateIOU(
            [base_left, base_left + base_width, base_top, base_top + base_height],
            [left, left + width, top, top + height],
        )
        
        if iou > max_iou:
            max_iou = iou
            matched_face_id = face_obj['id']
            
    if max_iou > 0.5:        
        for k, obj in enumerate(faces_collector):
            if obj['id'] == matched_face_id:
                faces_collector[k]['bytes'] = compare_face_bytes
                faces_collector[k]['attributes'] = attributes
                faces_collector[k]['face'] = current_face
                return True, copy.deepcopy(faces_collector[k])

    # 如果基于面积评估追踪失败，则进行人脸比对操作，比较费时
    for idx, face_obj in enumerate(faces_collector):
        if same_face(face_obj['bytes'], compare_face_bytes):
            faces_collector[idx]['bytes'] = compare_face_bytes
            faces_collector[idx]['attributes'] = attributes
            faces_collector[idx]['face'] = current_face
            return True, copy.deepcopy(faces_collector[idx])
    
    return False, None


frames_root_dir = './raw_volvo_demo_video_frames/'
results_root_dir = './face_detection_results/'

# 存储不同的人的信息，包括其ID和最新的某一帧中检测出的人脸图像
# faces_collector = list()
# face_id = 0
# faces_in_last_frame = list()

faces_collector = list()
start_index = 3320
pkls = [ele for ele in os.listdir(recog_result_root_dir) if (ele.endswith('.pkl') and (int(ele.split('.pkl')[0]) < start_index))]
pkls = sorted(pkls, key=lambda x:int(x.split('.')[0]))

for pkl_name in pkls:
    full_path = os.path.join(recog_result_root_dir, pkl_name)
    faces_list = pickle.load(open(full_path, 'rb'))
    
    face_ids_in_collector = [obj['id'] for obj in faces_collector]
    
    for obj in faces_list:
        if obj['id'] not in face_ids_in_collector:
            faces_collector.append(obj)

faces_in_last_frame = pickle.load(open(os.path.join(recog_result_root_dir, '{}.pkl'.format(start_index-1)), 'rb'))
max_face_id = -1
for face_obj in faces_in_last_frame:
    if face_obj['id'] > max_face_id:
        max_face_id = face_obj['id']
face_id = max_face_id + 1


# 每一帧中出现的人物信息，其元素为一个list
faces_distribution = list()

frames = [ele for ele in os.listdir(frames_root_dir) if ele.endswith('.jpg')]
frames = sorted(frames, key=lambda x:int(x.split('.')[0]))


for index, frame_name in enumerate(frames):
    if index < start_index:
        continue

    print('Processing frame {}'.format(frame_name))
    
    # 加载原图和人脸检测（含各种属性）结果
    frame_full_path = os.path.join(frames_root_dir, frame_name)
    response_full_path = os.path.join(results_root_dir, frame_name.split('.')[0] + '.json')
    image = cv2.imread(frame_full_path, cv2.IMREAD_COLOR)
    response = json.load(open(response_full_path, 'r'))
    image_height = image.shape[0]
    image_width = image.shape[1]
    
    face_details = response['FaceDetails']

    # 保存当前帧里面出现的人物信息
    total_faces_in_current_frame = list() # 所有脸需要按照face_id从小到大排序
    new_faces_in_current_frame  = list()
        
    # 遍历当前帧中检测到的每一个人脸
    for face_idx in range(len(face_details)):
        attributes = face_details[face_idx]
        height = int(attributes['BoundingBox']['Height'] * image_height)
        left = int(attributes['BoundingBox']['Left'] * image_width)
        top = int(attributes['BoundingBox']['Top'] * image_height)
        width = int(attributes['BoundingBox']['Width'] * image_width)
        
        current_face = image[top:top+height, left:left+width] 
        
        margin = int(max(height, width) * 0.2)
        min_height = top-margin if top-margin >= 0 else 0
        max_height = top+height+margin if top+height+margin <= image_height else image_height
        min_width = left-margin if left-margin >= 0 else 0
        max_width = left+width+margin if left+width+margin <= image_width else image_width
        margin_current_face = image[min_height:max_height, min_width:max_width] 
        
        temp_path = 'face.jpg'
        cv2.imwrite(temp_path, margin_current_face)
        with open(temp_path, 'rb') as rf:
            current_face_bytes = rf.read()
        
        if len(faces_collector) == 0:
            new_faces_in_current_frame.append({
                'face': current_face,
                'bytes': current_face_bytes,
                'id': face_id,
                'attributes': attributes
            })
                
            face_id += 1
        else:   
            already_appear, face_obj = identify_already_appear_before(
                faces_collector, 
                faces_in_last_frame,
                current_face_bytes, 
                current_face, 
                image_height,
                image_width,
                attributes)
            
            if already_appear:
                total_faces_in_current_frame.append(face_obj)
            else:
                new_faces_in_current_frame.append({
                    'face': current_face,
                    'bytes': current_face_bytes,
                    'id': face_id,
                    'attributes': attributes
                })

                face_id += 1
    
    total_faces_in_current_frame.extend(new_faces_in_current_frame)
    total_faces_in_current_frame = sorted(total_faces_in_current_frame, key=lambda x:x['id'])
    faces_in_last_frame = copy.deepcopy(total_faces_in_current_frame)        
        
    faces_collector.extend(new_faces_in_current_frame)
    faces_distribution.append(total_faces_in_current_frame)
    
    print(len(faces_collector), len(total_faces_in_current_frame), len(new_faces_in_current_frame))
    
    faces_record_dump_path = os.path.join(recog_result_root_dir, '{}.pkl'.format(frame_name.split('.')[0]))
    with open(faces_record_dump_path, 'wb') as wf:
        pickle.dump(total_faces_in_current_frame, wf)


Processing frame 3320.jpg
554 3 0
Processing frame 3321.jpg
554 3 0
Processing frame 3322.jpg
558 5 4
Processing frame 3323.jpg
558 5 0
Processing frame 3324.jpg
558 5 0
Processing frame 3325.jpg
558 5 0
Processing frame 3326.jpg
558 5 0
Processing frame 3327.jpg
558 5 0
Processing frame 3328.jpg
558 5 0
Processing frame 3329.jpg
558 5 0
Processing frame 3330.jpg
558 5 0
Processing frame 3331.jpg
558 5 0
Processing frame 3332.jpg
559 5 1
Processing frame 3333.jpg
559 4 0
Processing frame 3334.jpg
560 5 1
Processing frame 3335.jpg
561 6 1
Processing frame 3336.jpg
561 5 0
Processing frame 3337.jpg
561 5 0
Processing frame 3338.jpg
561 4 0
Processing frame 3339.jpg
562 5 1
Processing frame 3340.jpg
562 4 0
Processing frame 3341.jpg
563 5 1
Processing frame 3342.jpg
563 4 0
Processing frame 3343.jpg
563 4 0
Processing frame 3344.jpg
564 5 1
Processing frame 3345.jpg
564 5 0
Processing frame 3346.jpg
564 5 0
Processing frame 3347.jpg
564 5 0
Processing frame 3348.jpg
564 5 0
Processing fra